## Team 12 - Panzer Blow               
- Vikrant Dewangan - 2018111024             
- Shivaan Sehgal - 2018111026           
### Part A   
Disparity map generation was done by Shivaan , point cloud generation was done by Vikrant.
Motion and PnP was done by Vikrant, the plotting was done by Shivaan.
### Part B
First part was answered by Shivaan, Second part (one on the function) was done by Vikrant.

### NOTE - Please maintain the folder structure as follows - 
```
- parent folder
    - 12_panzer-blow.ipynb
    - data
        - img1
        - img2
        - poses.txt
        - calib.txt
```
Upon running it will create the output images as follows - 
```
- parent folder
    - 12_panzer-blow.ipynb
    - data
        - img1
        - img2
        - poses.txt
        - calib.txt
    - output
        - img460
        - img461
        ...
        - img480
        - point_cloud.pcd
```

# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [260]:
#import libraries:
import numpy as np 
# from sklearn.preprocessing import normalize #normalizing gives better results. Experiment with this
import cv2
from math import cos, sin
import open3d as o3d
import sys
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import normalize

In [261]:
def read_transformations(filename='./data/poses.txt'):
    poses = []
    with open("data/poses.txt","r") as f:
        lines = f.readlines()
        for line in lines:
            row = line.split(" ")
            row = np.array(row).astype('float64')
            poses.append(row.reshape((3, 4)))
    return poses

Provide explanation in this cell: 


In [262]:
def generateDisparity(imgL, imgR):
    
    imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    window_size = 5
    min_disp = -39
    num_disp = 144
    stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
        numDisparities = num_disp,
        disp12MaxDiff = 1,
        blockSize=5,
        P1=8 * 3 * window_size ** 2,    
        P2=32 * 3 * window_size ** 2,
        uniquenessRatio = 10,
        speckleWindowSize = 100,
        speckleRange = 32,
        preFilterCap=63
        )
    
    
    disparity = stereo.compute(imgL, imgR).astype(np.float32) / 16.0
    disparity = (disparity-min_disp)/num_disp
    return disparity

In [263]:
def project_points(points, colors, K, P, width, height):
    
    R = P[:, :3]
    R[:, 1] *= -1
    t = P[:, 3]

    dist_coeff = np.zeros((4, 1))
    projected, _ = cv2.projectPoints(points, R, t, K, dist_coeff)

    ones_row = np.ones((points.shape[0], 1))

    
    points = np.concatenate((points, ones_row), axis = 1)
    
    camera_projected = P @ points.T
    camera_projected = camera_projected.T.astype(np.int)
    
    xy = projected.reshape(-1, 2). astype(np.int)
    mask = (
        (0 <= xy[:, 0]) & (xy[:, 0] < width) &
        (0 <= xy[:, 1]) & (xy[:, 1] < height)
    )
    
    img_coord = xy[mask]
    img_color = colors[mask]
    
    camera_color = colors
    
    return camera_projected, camera_color, img_coord, img_color

In [264]:
def get_individual_disparities(disparity):
    individual_disparities = []
    x, y = disparity.shape
    
    for i in range(x):
        for j in range(y):
            individual_disparities.append([j, i, disparity[i, j], 1])
            
    return np.array(individual_disparities)

In [265]:
# Custom reproject function
def reprojectImageTo3D(Q, individual_disparities, img):
    points = Q @ individual_disparities.T
    points /= points[3, :]
    points = points[:3, :]
    points = points.T
    
    output = np.zeros((img.shape[0], img.shape[1], 3))
    i = 0
    j = 0
    
    for p in range(points.shape[0]):
        output[i][j] = points[p]
        j += 1
        
        if j >= 1226:
            j = 0
            i += 1
        
    
    return output

In [266]:
def getImage(points, color, height, width):
    image = np.zeros((height, width, 3), dtype=color.dtype)
    image[points[:, 1], points[:, 0]] = color
    return image

## Disparity Map Generation

In [267]:
images_left = []
images_right = []
for i in range(60,81):
    images_left.append(cv2.cvtColor(cv2.imread('data/img2/00000004' + str(i) + '.png'), cv2.COLOR_BGR2RGB))
    images_right.append(cv2.cvtColor(cv2.imread('data/img3/00000004' + str(i) + '.png'), cv2.COLOR_BGR2RGB))

In [268]:
baseline = 0.53790448812
K = np.array([[7.070912e+02, 0.000000e+00, 6.018873e+02], [0.000000e+00, 7.070912e+02, 1.831104e+02], [0.000000e+00, 0.000000e+00, 1.000000e+00]])
focal_length = K[0][0]
cx= K[0][2]
cy= K[1][2]
poses = read_transformations()

# Formation of Q matrix to reproject back into 3d
Q = np.float32([[1, 0, 0, -0.5*width],
                [0,-1, 0,  0.5*height], 
                [0, 0, 0, -focal_length], 
                [0, 0, -1/baseline,  0]])


In [269]:
all_pts = np.array([]).reshape(0,3)
all_colors = np.array([]).reshape(0,3)

for i in range(460, 481):
    print("i  = \t",i);
    image_left = images_left[i - 460]
    image_right = images_right[i - 460]

    disparity = generateDisparity(image_left, image_right)
    parallax_map = get_individual_disparities(disparity)
    points_3D = reprojectImageTo3D(Q, parallax_map, image_left)
#     points_3D = cv2.reprojectImageTo3D(disparity, Q)
    
    mask_map = disparity > disparity.min()

    colors = cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB)

    output_points = points_3D[mask_map]
    output_colors = colors[mask_map]
    
    #print(output_colors[1])
    camera_projected, camera_color, img_coords, img_color = project_points(output_points, output_colors, K, poses[i-460], l, w)
    
    all_pts = np.concatenate((all_pts, camera_projected), axis=0)
    all_colors = np.concatenate((all_colors, camera_color), axis=0)
    
    img = getImage(img_coords, img_color, w, l)
    cv2.imwrite("output/img" + str(i) + ".png", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    


i  = 	 460
i  = 	 461
i  = 	 462
i  = 	 463
i  = 	 464
i  = 	 465


<ipython-input-265-1820edf7b278>:4: RuntimeWarning: divide by zero encountered in true_divide
  points /= points[3, :]
<ipython-input-265-1820edf7b278>:4: RuntimeWarning: invalid value encountered in true_divide
  points /= points[3, :]
<ipython-input-263-773339118382>:15: RuntimeWarning: invalid value encountered in matmul
  camera_projected = P @ points.T


i  = 	 466
i  = 	 467
i  = 	 468
i  = 	 469
i  = 	 470
i  = 	 471
i  = 	 472
i  = 	 473
i  = 	 474
i  = 	 475
i  = 	 476
i  = 	 477
i  = 	 478
i  = 	 479
i  = 	 480


In [270]:
mask = (
        (-3000 <= all_pts[:, 0]) & (all_pts[:, 0] < 3000) &
        (-3000 <= all_pts[:, 1]) & (all_pts[:, 1] < 3000) & 
        (-900 <= all_pts[:, 2]) & (all_pts[:, 2] < 900)
    )

plot_pts = all_pts[mask]
plot_pts[:, 1] *= -1
plot_colors = (all_colors[mask]/255.0).astype(float)

pcd = o3d.geometry.PointCloud()
vis = o3d.visualization.Visualizer()
pcd.points = o3d.utility.Vector3dVector(plot_pts) #numpy_points is your Nx3 cloud
pcd.colors = o3d.utility.Vector3dVector(plot_colors) #numpy_colors is an Nx3 matrix with the corresponding RGB colors

o3d.visualization.draw_geometries([pcd])

In [271]:
o3d.io.write_point_cloud("output/point_cloud.pcd",pcd)

True

----
### PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

In [272]:
def readFile(file):
	pcd = o3d.io.read_point_cloud(file)
	points = np.asarray(pcd.points)
	colors = np.asarray(pcd.colors)
	return pcd, points, colors

In [273]:
def projectToImage(K, P, points, colors, height, width):
    R = P[:, :3]
#     R[:, 1] *= -1
    t = P[:, 3]

    dist_coeff = np.zeros((4, 1))
    projected, _ = cv2.projectPoints(points, R, t, K, dist_coeff)
    projected = projected.reshape(-1, 2). astype(np.int)
    
    mask = (
        (0 <= projected[:, 0]) & (projected[:, 0] < width) &
        (0 <= projected[:, 1]) & (projected[:, 1] < height)
    )


    img_coord = projected[mask]
    img_color = colors[mask]
    
    return img_coord, img_color

In [274]:
def DLT(x,X):
    """
    your code here
    """
    matrix = []
    for i in range(len(X)):
        # w1 = X[i][0]
        # w2 = X[i][1]
        # w3 = X[i][2]
        # i1 = x[i][0]
        # i2 = x[i][1]
        #row1 = [-w1,-w2,-w3,-1,0,0,0,0,i1*w1,i1*w2,i1*w3,i1]
        #row2 = [0,0,0,0,-w1,-w2,-w3,-1,i2*w1,i2*w2,i2*w3,i2]

        row1 = [-X[i][0],-X[i][1],-X[i][2],-1,0,0,0,0,x[i][0]*X[i][0],x[i][0]*X[i][1],x[i][0]*X[i][2],x[i][0]]
        row2 = [0,0,0,0,-X[i][0],-X[i][1],-X[i][2],-1,x[i][1]*X[i][0],x[i][1]*X[i][1],x[i][1]*X[i][2],x[i][1]]

        matrix.append(row1)
        matrix.append(row2)

	# print(matrix)
    u, s, vh = np.linalg.svd(matrix, full_matrices=False)
	# print(vh)
    vh = vh.transpose()
    P_vec = []
    for i in range(12):
        P_vec.append(vh[i][11])

	# print(P_vec)
    P_mat = []
    P_mat.append(P_vec[0:4])
    P_mat.append(P_vec[4:8])
    P_mat.append(P_vec[8:12])
    P = P_mat
    return np.array(P)

In [275]:
pcd, points, colors = readFile("output/point_cloud.pcd")
# show(pcd)
print(points.shape)
print(colors.shape)

(5643511, 3)
(5643511, 3)


In [276]:
poses = read_transformations()
P = poses[0]
K = np.array([[7.070912e+02, 0.000000e+00, 6.018873e+02], [0.000000e+00, 7.070912e+02, 1.831104e+02], [0.000000e+00, 0.000000e+00, 1.000000e+00]])
height = 370
width = 1226

## 1. Generating 2D-3D correspondences between point cloud and image
We first take the Q matrix from the R and t matrix we got above. Then we multiply K with Q to get our correct P matrix. Then we sample randomly 100 points from our point cloud

In [277]:
# Correct R|t Matrix 
Q = np.array([ [-9.1e-01, 5.5e-02, -4.2e-01, -1.9e+02],
               [4.2e-02, 9.983072e-01, 4.2e-02, 1.7e+00],
               [4.2e-01, 2.1e-02, -9.2e-01, 5.5e+01] ])

row = [0,0,0,1]
row = np.array(row)
Q=np.vstack((Q,row))

Q=np.linalg.inv(Q)
Q = Q[:3, :]


# Correct projection matrix, the one we should be getting after GN 
P = np.matmul(K, Q) # 3x4 matrix 
print("Correct projection matrix: \n", P)

Correct projection matrix: 
 [[-8.88191631e+02  5.41139423e+01 -2.46276119e+02 -1.55303217e+05]
 [-3.62607430e+01  7.13441345e+02 -1.49909078e+02  1.42607815e+02]
 [-4.12056509e-01  4.15693433e-02 -8.96946037e-01 -2.90293725e+01]]


In [278]:
k = 100
random_3d = np.array(random.choices(points, k=k))

row = np.ones((k, 1))

random_3d = np.concatenate([random_3d, row], axis = 1)
random2d_image = (P @ random_3d.T).T

for i in range(k):
    random2d_image[i] /= random2d_image[i, 2]  # last element becomes 1

print(random_3d.shape) # 100 times 4
print(random2d_image.shape) # 100 times 3

(100, 4)
(100, 3)


## Step 2 : Computing the projection error

In [279]:
pts_2d = []
for pt_3d in random_3d:
    pt_2d = np.matmul(P, pt_3d.T)
    pt_2d = pt_2d/pt_2d[2]
    pts_2d.append(pt_2d)

error = random2d_image - pts_2d # 100 x 3
error = np.delete(error, 2, axis=1) # 100 x 2 i.e. 100 points and 2 coordinates for each 
#     print("error:", error)
sum_sq_error = 0
for obj in error:
    sq_error = np.matmul(obj.T, obj)
    sum_sq_error = sum_sq_error + sq_error
print("squared sum error:\t", sum_sq_error)

# print(P/P[2][3])
P_calc = DLT(random2d_image,random_3d)
P_calc = np.linalg.inv(K) @ P_calc 
print("P from DLT : \n",P_calc/P_calc[2][3])

P = P_calc
total = 0
for i in range(len(random_3d)):
    px = P @ random_3d[i]
    px /= px[2]
    error = random2d_image[i] - px
    total += error @ error.T
#     print(error)
print("Total error after reconstruction : \t",total)

squared sum error:	 1.0316747077345615e-24
P from DLT : 
 [[ 3.11881099e-02 -1.41738750e-03 -1.43027570e-02  6.71480208e+00]
 [-1.90930035e-03 -3.43864043e-02 -6.98176993e-04 -2.65910444e-01]
 [ 1.41944683e-02 -1.43197526e-03  3.08978789e-02  1.00000000e+00]]
Total error after reconstruction : 	 100502540.8094665


## Step 3 : Using Gauss Newton for pose $T_k$

In [280]:
def gaussNewton(lr=1,P_est=np.array([ [-8.8e+02,5.4e+01,-2.4e+02,-1.5e+05],[-3.6e+01,7.1e+02,-1.4e+02,1.4e+02],[-4.1e-01,4.1e-02,-8.9e-01,-2.9e+01] ])):
    print("Running Gauss newton: \n ")
    sum_sq_error = 10000
    pts_3d = random_3d
    actual_pts_2d  = random2d_image
    while True: # number of iterations for the algorithm, to be updated later
        prev_error = sum_sq_error
        pts_2d = [] # corresponding 2D location estimated based on our P matrix 
        J = []
        for pt_3d in pts_3d:
            pt_2d = np.matmul(P_est, pt_3d)
            pt_2d = pt_2d/pt_2d[2]
            pts_2d.append(pt_2d)

        error = actual_pts_2d - pts_2d # 100 x 3
        error = np.delete(error, 2, axis=1) # 100 x 2 i.e. 100 points and 2 coordinates for each 
    #     print("error:", error)
        sum_sq_error = 0
        for obj in error:
            sq_error = np.matmul(obj.T, obj)
            sum_sq_error = sum_sq_error + sq_error

        print("squared sum error:", sum_sq_error)
        if(abs(sum_sq_error - prev_error) < 0.0001):
            break

        #calculating J
        for pt_3d in pts_3d:

            X = P_est[0][0]*pt_3d[0] + P_est[0][1]*pt_3d[1] + P_est[0][2]*pt_3d[2] + P_est[0][3]
            Y = P_est[1][0]*pt_3d[0] + P_est[1][1]*pt_3d[1] + P_est[1][2]*pt_3d[2] + P_est[1][3]
            Z = P_est[2][0]*pt_3d[0] + P_est[2][1]*pt_3d[1] + P_est[2][2]*pt_3d[2] + P_est[2][3]

            X1 = pt_3d[0]
            Y1 = pt_3d[1]
            Z1 = pt_3d[2]

            J1 = X1/Z
            J2 = Y1/Z
            J3 = Z1/Z
            J4 = 1/Z
            J5 = 0
            J6 = 0
            J7 = 0
            J8 = 0
            J9 = (-X*X1)/(Z*Z)
            J10 = (-X*Y1)/(Z*Z)
            J11 = (-X*Z1)/(Z*Z)
            J12 = -1/(Z*Z)

            J13 = 0
            J14 = 0
            J15 = 0
            J16 = 0
            J17 = J1
            J18 = J2
            J19 = J3
            J20 = J4
            J21 = (-Y*X1)/(Z*Z)
            J22 = (-Y*Y1)/(Z*Z)
            J23 = (-Y*Z1)/(Z*Z)
            J24 = J12
    
            J = np.append(J,np.array([J1,J2,J3,J4,J5,J6,J7,J8,J9,J10,J11,J12,J13,J14,J15,J16,J17,J18,J19,J20,J21,J22,J23,J24]))
        
#         print(J.shape)
        J = J.reshape(200,12)

        JTJ = np.matmul(J.T, J)
        JTJ = np.linalg.pinv(JTJ) # 12 x 12

        error = np.reshape(error, (200,1))
        e = np.matmul(J.T, error) # 12x1

        deltaP = np.matmul(JTJ, e)
        deltaP = deltaP.reshape(3,4)
        P_est = P_est + lr * deltaP

    print("Predicted projection matrix after Gauss Newton: \n")
    print(P_est)
    return P_est

In [281]:
P_gauss = gaussNewton()
print(P_gauss/P_gauss[2][3])

Running Gauss newton: 
 
squared sum error: 16964.85474082398
squared sum error: 4754.293672051491
squared sum error: 3635.9736586956233
squared sum error: 2552.7442156459624
squared sum error: 2067.1158407525727
squared sum error: 1480.8405556271775
squared sum error: 1184.4757625904124
squared sum error: 858.3845458875402
squared sum error: 681.1759603276668
squared sum error: 497.79711088010504
squared sum error: 393.4328564462422
squared sum error: 289.29381576974544
squared sum error: 228.5942592187186
squared sum error: 168.91597518769933
squared sum error: 134.01105683030798
squared sum error: 99.50895171503458
squared sum error: 79.66603840081068
squared sum error: 59.53857679228073
squared sum error: 48.402611191404006
squared sum error: 36.54697285857329
squared sum error: 30.396141315863602
squared sum error: 23.337735739194997
squared sum error: 20.012245026052646
squared sum error: 15.758861308075675
squared sum error: 14.015754077578917
squared sum error: 11.4173141132911

# Part B

# Project 2 - Part B:

## 1. SfM pipeline (`6 mark`)

To get the context of below questions, take a look at the code above: The same questions have been asked at different places above as comments in the code.

1. `0.5 mark` **Basics** - How do we know this (`camera_ind`) information in practical setting? In other words, how do we know observations in `points_2d` belong to which camera. Explain. 
    - Ans-1 - Basics: We have `n_camera` cameras and `n_observations` different image points. This is established via the individual view of the object from different dimensions. From the first few views, we should be able to decipher which image features is common across these observations using SIFT or BRIEF algorithm. We can map them to the 3D world, and then subsequently obtain our Essential matrix  $\hat{E}$ and our Fundamental matrix  $\hat{F}$ from them. We can now estimate the pose of our calibrated camera using PnP algorithm. Thus, for each of remaining our given `n_camera` cameras and points_2d, we can estimate which view corresponded to which camera that is `camera_ind`.
2. `0.5 mark` **Basics** - How do we know this (`point_ind`) information in practical setting?  In other words, how do we know observations in `points_2d` belong to which 3D point. Explain.
    - Ans-2 - Basics: Each camera gives some observations, and we can obtain the correlation of each point to camera from this view. We obtain the pose of camera matrix using PnP algorithm, and from this, we can have a **metric** reconstruction of the `points_2d` to 3D world. Thus we would get pairs of 2D points with their corresponding 3D point.
3. `0.5 mark` **Transformations** - `project()` function: In the `project()` function, would it make any difference if I do translate first, then rotate? Why/why not?
    - Ans-3 - Transformations: Yes, doing translation first and then rotation would make major difference, as translation is not given with respect to the world frame but the camera frame. And if we bring translation to world frame, the equation would completely change - 
    
    Original equation - 
    $$
    {}^cX = R^{c}_w {}^wX + {}^ct_w^c
    $$
    If we instead have translation first (after converting to world frame) - 
    $$
    {}^cX = R^{c}_w ({}^wX + T^{c}_w\cdot{}^ct_w^c) = R^{c}_w {}^wX + R^{c}_w T^{c}_w\cdot{}^ct_w^c
    $$    
4. `0.5 mark` **Jacobian** - `bundle_adjustment_sparsity()` function: m above is not "M*N" (*2) unlike our lecture notes. Why is that so?
    - Ans-4 - Jacobian: (Assuming here 2MN is meant as Jacobian has twice the number of rows of points, as each point gives 2 residuals). Why is our number of points not exactly N is because in our notes, we assumed that each image would give us exactly N points, which might not be the case. We may have more than or less than the points per image. The correct size is given by `camera_indices.size`.
5. `2 mark` **Jacobian & Parameters** - `bundle_adjustment_sparsity()` function: 
    1.  Why are we doing `n_cameras * 9` here instead of `n_cameras * 12`? Recollect: Every individual motion Jacobian was (1*)12 in our lecture notes. 
        - Ans 5.1 - Jacobian & Parameters: In Standard approach, our camera matrix consists of 12 parameters ($3 \times 4$) to optimize, which is why in their case, we have         
        `n = n_cameras * 12 + n_points * 3`
        
             But in our case, we have 
             
         `n = n_cameras * 9 + n_points * 3`
         
         The reason for this is because we are using axis-angles and thus only 9 parameters to optimize over here - 3 each for rotation (axis-angles) and translation, 2 for distortion and 1 for focal length. 
    2. Ignoring the scale parameters, what was the number of unknown parameters in our lecture notes in terms of `n_cameras` and `n_points`? What is it here in the code? Is it different? If so, what is and why? [Link of notes](https://www.notion.so/Stereo-Structure-from-Motion-9fdd81e4194f4803ac9ba7552df56470).
        - Ans 5.2 - Jacobian & Parameters:
The number of unknown parameters was `12 x n_cameras + 3 x n_points` in the lectures, as we  also optimized over our 3x4 projection matrix, essentially optimizing for all the intrinsics and extrinsic parameters. Since we do not have the rotation matrix in the 3D rotation group  but in standard Euclidean space, we actually have the told instead of `6 x n_cameras + 3x n_points parameter`. Since we are using the axis- angle rotations here, we have only 3 rotation(axis-angle) parameters in addition to  translation parameters, and 3 intrinsic camera parameters.Thus we have `9x n_cameras + 3xn_points parameters`.        
6. `2 mark` **Sparsity, Residual Vector & Jacobian** - `bundle_adjustment_sparsity()` function: Explain what you understand from above 6 lines of code by coding a simple toy example yourself to illustrate how it is different from what you've learnt in class. ([Coding toy example + elaborating in words]- both are compulsory.)
    - Ans 6 - Sparsity, Residual Vector & Jacobian: Structure of the Jacobian) The biggest difference is that here we have `2 * M * camera_ind.size ` rows, and that rotation matrix is from axis angles. Another difference is that the residuals in the class were ordered first according to the points and then according to images - while here, they appear in no particular order, which is alright for as long as one order is consistently maintained. 
    ```
    for s in range(9):
        A[2 * i, camera_indices * 9 + s] = 1
        A[2 * i + 1, camera_indices * 9 + s] = 1

    for s in range(3):
        A[2 * i, n_cameras * 9 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 9 + point_indices * 3 + s] = 1
    ```
    The above 6 lines of code shall be explained one by one. First we have A is a ```lil_matrix`` from a sparse object. It is more efficient this way. We know that J is very sparse and that there are somr elements which are not always 0. We initially set all the 9 columns corresponding to the parameters of the corresponding camera are set to 1. In  the next for loop, we sets all the 3 columns correspondings to world points involved in the ith observation to 1.
    

## 2. Initializing R,t and 3D points for SfM given 2 images (`4 mark`)

Using OpenCV functions, mention how you would initialize R,t (poses) and 3D points for SfM given 2 images and K matrix. You don't need to implement it, just mention function names with input/output arguments clearly and briefly explain what they do (You don't need to give detailed answers). A sample answer could be as follows:

**Ans 2:**

1. First, we do features mathcing using the following function calls:
    ```
    sift = cv2.SIFT_create() 
    key1,dec1 = sift.detectAndCompute(img1,None)
    key2,dec2 = sift.detectAndCompute(img2,None)
     ```
    This returns the keypoints and their correspondin descriptions for the tow images. 
    Now, we match the corresponding features between the two images, using BF matcher.
    ```
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)     ```
    After this, the matching points from within matches are seprated into points in the first image(pts1) and pois in the second image(pts2) as follows:
     ```
    pts1 = np.array([key1[m[0].queryIdx].pt for m in matches])
    pts2 = np.array([key2[m[0].trainIdx].pt for m in matches])
     ```
 
2. Now that we have the corresponding points , we find the essential matrix using ```cv2.findEssentialMatrix```.
    The parameters provided are the lists of matching points, pts1 and pts2 as well as the camera intrensics, K.
    The methored can be set to ```cv2.RANSAC```, and RANSAC''s theshhold and probablity values can be set.
 
3. Now that we have essential matrix, we can decompose it into R,t as following.
    ```
    Rt=cv2.recoverpose(E,pts1,pts2,K). This takes Essential matrix, K matrix and the corresponding points in 2 images as input arguments and gives us R,t.
    Rt[1] will give R anf Rt[2] will give t.
     ```
4. Now that we have teh cameras extrensic(R and t), we find 3d points as follows:
    (This calculated the points in the camers frame)
     ```
    P1 = K @ np.hstack((np.eye(3),np.zeros((3,1))))
    P2 = K @ np.hstack((R,t))
    ptsW = cv2.triangulatePoints(P1,P2,pts1,pts2)
    PtsW /= ptsW[1]
    ptsW[:3]
    ```